# Libraries

In [32]:
# Basic libraries
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mstats

# Working with files
import os
import glob
import io


# Working with OpenML dataset
import openml
from openml.datasets import edit_dataset, fork_dataset, get_dataset


# Feature Selection
from sklearn.feature_selection import RFE
from sklearn.preprocessing import LabelEncoder, StandardScaler, LabelBinarizer, OneHotEncoder
from sklearn.model_selection import StratifiedKFold

# Scalers
from sklearn.preprocessing import MinMaxScaler, StandardScaler


# Performance Metrics
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, make_scorer
from sklearn.metrics import f1_score


# Classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier

# NN
import torch
import torch.nn as nn
from torch.utils.data import random_split, DataLoader, TensorDataset
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


import warnings
warnings.filterwarnings('ignore')

# Functions

In [2]:
def featureEnginering(df, train = False):
    
    if train == True:
        
        ##### Checking for each column the type and the number of distinct and similar values (also see if there are missing values)
        missing_col = []

        for col in df.columns:
            if df[col].isna().sum() > 0:
                missing_col.append(col)

        ##### Filling missing values
        for col in missing_col:
            median_value = df[col].median()
            df[col].fillna(median_value, inplace = True)


        ##### Treating Outliers
        values_to_exclude = ['Average_ACT_Score']
        filtered_col = [x for x in missing_col if x not in values_to_exclude]

        for col in missing_col:

            # Get the minimum and maximum values in the column
            min_value = df[col].min()
            max_value = df[col].max()

            # Adjust k based on the range of values
            if max_value < 100:
                pass

            else:
                if 100 <= max_value <= 700:
                    k = 0.8
                elif 700 <= max_value <= 5000:
                    k = 1.7 
                else:
                    k = 2.5

                # Calculate the interquartile range (IQR)
                Q1 = df[col].quantile(0.25)
                Q3 = df[col].quantile(0.75)
                IQR = Q3 - Q1


                # Lower and upper bounds
                lower_bound = Q1 - k * IQR
                upper_bound = Q3 + k * IQR

                df[col] = df[col].clip(lower = lower_bound, upper = upper_bound)


        ##### Mapping Categorical Features
        states = list(set(df['State']))
        mapped_states = {st: idx if st is not None else -1 for idx, st in enumerate(states)}
        df['State'] = df['State'].replace(mapped_states)
        
        
        target = {'N': 0, 'P': 1}
        df['binaryClass'] = df['binaryClass'].map(target)


        ##### Scaling
#         scaler = StandardScaler()
#         scaler.fit(X_train)

#         X_train_scaled = scaler.transform(X_train)
    
    
    return df

In [3]:
class Dataset:
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype = torch.float32)
        self.labels = torch.tensor(labels, dtype = torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [4]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, output_size, layer_sizes, activation_funcs):
        super(SimpleNN, self).__init__()

        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()
        self.dropouts = nn.ModuleList()
        self.activation_funcs = activation_funcs

        self.batch_norm1 = nn.BatchNorm1d(input_size)
        # input layer
        self.layers.append(nn.utils.weight_norm(nn.Linear(input_size, layer_sizes[0])))
        self.batch_norms.append(nn.BatchNorm1d(layer_sizes[0]))
        self.dropouts.append(nn.Dropout(0.2))

        # Hidden layers
        for i in range(1, len(layer_sizes)):
            self.layers.append(nn.utils.weight_norm(nn.Linear(layer_sizes[i-1], layer_sizes[i])))
            if i < len(layer_sizes) - 1:
                self.batch_norms.append(nn.BatchNorm1d(layer_sizes[i]))
            self.dropouts.append(nn.Dropout(0.5))

        # Output layer
        self.output = nn.utils.weight_norm(nn.Linear(layer_sizes[-1], output_size))


    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropouts[0](x)
        x = self.layers[0](x)
        x = self.activation_funcs[0](x)

        for i in range(1, len(self.layers)):
            if i < len(self.batch_norms):
                x = self.batch_norms[i](x)
            x = self.dropouts[i](x)
            x = self.layers[i](x)
            if i < len(self.activation_funcs):
                x = self.activation_funcs[i](x)

        x = self.output(x)
        return x

In [5]:
def grid_search(configs, dataset):

    best_config = None
    best_score = None
    
    scores = []
#     best_scores = []
    layers = []
    activations = []

    for config in configs:
        best_score = 0
        layer_size = config['layer_sizes']
        activation = config['activations']

        model = SimpleNN(
            input_size = INPUT_SIZE, output_size = OUTPUT_SIZE,
            layer_sizes = config['layer_sizes'],  activation_funcs = config['activations']
        )

        print(f"Layer sizes: {layer_size}, Activations: {[type(act).__name__ for act in activation]}")

        score, loss, all_f1_scores = train_and_evaluate_model(model, dataset)

        if score > best_score:
            best_score = score
            best_config = config
            
        scores.append(all_f1_scores)
#         best_scores.append(all_best_losses)
        layers.append(layer_size)
        activations.append([type(act).__name__ for act in activation])

    print(f"Best F1 Score: {best_score:.2f}, Layer sizes: {layer_size}, Activations: {[type(act).__name__ for act in activation]}")
    
    
    
    
    return best_config, best_score, scores, layers, activations

In [6]:
def local_search(base_config, dataset, iterations=10):
    def get_neighbors(configuration):
        neighbors = []

        # Example: Add a layer
        neighbors.append({
            'layer_sizes': configuration['layer_sizes'] + [64],
            'activations': configuration['activations'] + [nn.ReLU()]
        })

        # Example: Remove a layer
        if len(configuration['layer_sizes']) > 1:
            neighbors.append({
                'layer_sizes': configuration['layer_sizes'][:-1],
                'activations': configuration['activations'][:-1]
            })
        return neighbors

    def get_model(config):
        layer_size = config['layer_sizes']
        activation = config['activations']
        model = SimpleNN(INPUT_SIZE, OUTPUT_SIZE, layer_size, activation)

        return model

    current_configuration = base_config
    model = get_model(current_configuration)
    current_score, current_loss, _ = train_and_evaluate_model(model, dataset)

    while True:
        neighbors = get_neighbors(current_configuration)
        any_improvement = False

        for neighbor_config in neighbors:
            neighbor_model = get_model(neighbor_config)
            score, loss, _ = train_and_evaluate_model(neighbor_model, dataset)

            if score > current_score:
                current_configuration = neighbor_config
                current_score = score
                any_improvement = True
                break  # Move to the first improving neighbor

        if not any_improvement:
            break  # No improvement found, stop the search

    return current_configuration, current_score


In [7]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader):
    model.train()
    final_loss = 0

    for data, target in dataloader:
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target.long())
        loss.backward()
        optimizer.step()
        scheduler.step()

        final_loss += loss.item()
    final_loss /= len(dataloader)

    return final_loss

In [8]:
def valid_fn(model, loss_fn, dataloader):
    model.eval()
    final_loss = 0
    final_f1_score = 0

    for data, target in dataloader:
        outputs = model(data)
        loss = loss_fn(outputs, target.long())
        final_loss += loss.item()

        preds = outputs.argmax(dim=1).detach().cpu().numpy()
        final_f1_score += (f1_score(target.cpu().numpy(), preds.round(), average='weighted'))

    final_f1_score /= len(dataloader)
    final_loss /= len(dataloader)

    return final_loss, final_f1_score

In [9]:
# Function to create and train a network
def train_and_evaluate_model(model, dataset, n_splits = 5):


    labels = [label for _, label in dataset]
    kfold = StratifiedKFold(n_splits = n_splits, shuffle=True)
    
    all_f1_scores = []

    for fold, (train_ids, valid_ids) in enumerate(kfold.split(range(len(dataset)), labels)):

        # Splitting the dataset
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
        valid_subsampler = torch.utils.data.SubsetRandomSampler(valid_ids)

        # Creating data loaders
        train_loader = DataLoader(dataset, batch_size = BATCH_SIZE, sampler = train_subsampler)
        valid_loader = DataLoader(dataset, batch_size = BATCH_SIZE, sampler = valid_subsampler)

        # initialization
        model.to(DEVICE)

        optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3,
                                  max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(train_loader))
        loss_fn = nn.CrossEntropyLoss()
        early_stopping_steps = EARLY_STOPPING_STEPS
        early_step = 0


        # Training loop
        best_loss = np.inf
        for epoch in range(EPOCHS):

            train_loss = train_fn(model, optimizer,scheduler, loss_fn, train_loader)
            valid_loss, score = valid_fn(model, loss_fn, valid_loader)

            if valid_loss < best_loss:
                best_loss = valid_loss
                f1_score = score

                torch.save(model.state_dict(), f"fold{fold}_.pth")

            elif(EARLY_STOP == True):
                early_step += 1

            if (early_step >= early_stopping_steps):
                break


        print(f"Completed training fold {fold}, and best f1 score is {f1_score:.2f}")
        all_f1_scores.append(f1_score)

    return f1_score, best_loss, all_f1_scores 

In [10]:
def load_best_model_and_predict(model, dataset, fold, model_path):
    # Load the best model for a specific fold
    model.load_state_dict(torch.load(model_path))
    model.to(DEVICE)
    model.eval()

    # Create DataLoader for the test set
    test_loader = DataLoader(dataset, batch_size = BATCH_SIZE)

    all_predictions = []
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(DEVICE)

            # Forward pass
            outputs = model(inputs)

            # Convert the model output to probabilities using softmax
            probabilities = torch.nn.functional.softmax(outputs, dim = 1)

            # Get the predicted class for each sample
            _, predictions = torch.max(probabilities, 1)

            # Convert predictions to a NumPy array and append to the list
            all_predictions.extend(predictions.cpu().numpy())

    return all_predictions

In [11]:
def calculate_f1_scores(all_fold_predictions, dataset):
    f1_scores = []

    for fold, predictions in enumerate(all_fold_predictions):
        # Assuming binary classification; adjust the average parameter if needed
        f1 = f1_score(dataset.labels, predictions, average = 'weighted')
        f1_scores.append(f1)

    return f1_scores

# Train

In [15]:
# Load the dataset
dataset = openml.datasets.get_dataset(930)
big_df, *_ = dataset.get_data()

# Feature engineering
processed_df = featureEnginering(big_df, train=True)

# Split the dataset into features and labels
feature_names = list(filter(lambda x: x not in ['binaryClass'], processed_df.columns))

X = processed_df[feature_names].values
y = processed_df['binaryClass'].values

# Create a Dataset object
dataset = Dataset(X, y)

In [16]:
# HyperParameters

INPUT_SIZE = len(processed_df[feature_names].columns)
OUTPUT_SIZE = len(processed_df['binaryClass'].unique())

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE = 'cpu'
EPOCHS = 25
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False

print(DEVICE)

cpu


## Simple NN

In [17]:
# Define the pipeline
def pipeline(dataset, configs):
    
    # Perform grid search
    best_configuration, best_score, scores, layers, activations = grid_search(configs, dataset)
    
    print('\n')
    print(f"Best Configuration for the model: {best_configuration}")
    print(f"Best F1-Score for the model: {round(100 * best_score, 2)}%")


    ## Creating Dataframe with results 
    grid_results_df = pd.DataFrame(columns = ['Layers', 'Activations'] + [f'Fold{i+1}' for i in range(5)])
    grid_results_df['Layers'] = layers
    grid_results_df['Activations'] = activations


    for i in range(0, 5):
        vals = [lst[i] for lst in scores]
        grid_results_df[f'Fold{i+1}'] = vals

#     print(grid_results_df)


    # Perform local search
    best_configuration = {'layer_sizes': [64, 64], 'activations': [nn.ReLU(), nn.ReLU()]}
    final_config, final_score = local_search(best_configuration, dataset)
    
    print(f"Final Configuration for the model: {final_config}")
    print(f"Final F1-Score for the model: {round(100 * final_score, 2)}%")

    # Train and evaluate the final model on the test set for all folds
    all_fold_predictions = []

    for fold in range(0, 5):  # Assuming 5 folds
        model_path = f"fold{fold}_.pth"
        print(model_path)
        
        model = SimpleNN(
            input_size = INPUT_SIZE, output_size=OUTPUT_SIZE,
            layer_sizes = final_config['layer_sizes'], activation_funcs = final_config['activations']
        )

        # Train the model
        train_and_evaluate_model(model, dataset)

        # Load the best model and make predictions
        predictions = load_best_model_and_predict(model, dataset, fold, model_path)

        # Append predictions to the list
        all_fold_predictions.append(predictions)
        
        
    # Calculate F1 scores for each fold
    f1_scores = calculate_f1_scores(all_fold_predictions, dataset)

    # Put F1 scores into a dataframe
    f1_scores_df = pd.DataFrame({'Fold': [f'Fold{i+1}' for i in range(5)], 'F1 Score': f1_scores})

    return grid_results_df, all_fold_predictions, f1_scores_df

In [18]:
dataset_id = 930

configs = [
                {'layer_sizes': [64], 'activations': [nn.ReLU()]},
                {'layer_sizes': [64, 64], 'activations': [nn.ReLU(), nn.ReLU()]},
                {'layer_sizes': [128, 64], 'activations': [nn.Tanh(), nn.ReLU()]},
                {'layer_sizes': [128, 64], 'activations': [nn.Tanh(), nn.Tanh()]},
                {'layer_sizes': [64, 64, 32], 'activations': [nn.ReLU(), nn.Tanh(), nn.ReLU()]},
                {'layer_sizes': [128, 128, 64], 'activations': [nn.ReLU(), nn.ReLU(), nn.ReLU()]},
                {'layer_sizes': [128, 128, 64], 'activations': [nn.Tanh(), nn.ReLU(), nn.ReLU()]}]

grid_results_df, pipeline_results, f1_scores_df = pipeline(dataset, configs)
# print(pipeline_results)
grid_results_df

Layer sizes: [64], Activations: ['ReLU']
Completed training fold 0, and best f1 score is 0.85
Completed training fold 1, and best f1 score is 0.82
Completed training fold 2, and best f1 score is 0.85
Completed training fold 3, and best f1 score is 0.87
Completed training fold 4, and best f1 score is 0.84
Layer sizes: [64, 64], Activations: ['ReLU', 'ReLU']
Completed training fold 0, and best f1 score is 0.84
Completed training fold 1, and best f1 score is 0.81
Completed training fold 2, and best f1 score is 0.84
Completed training fold 3, and best f1 score is 0.82
Completed training fold 4, and best f1 score is 0.86
Layer sizes: [128, 64], Activations: ['Tanh', 'ReLU']
Completed training fold 0, and best f1 score is 0.81
Completed training fold 1, and best f1 score is 0.78
Completed training fold 2, and best f1 score is 0.85
Completed training fold 3, and best f1 score is 0.83
Completed training fold 4, and best f1 score is 0.85
Layer sizes: [128, 64], Activations: ['Tanh', 'Tanh']
Com

RuntimeError: File fold4_.pth cannot be opened.

In [ ]:
f1_scores_df

## MLP

In [37]:
# Define the MLPClassifier
mlp = MLPClassifier(max_iter=100)

# Define the parameter grid for grid search
param_grid = {
    'classifier__hidden_layer_sizes': [(50,50), (100,), (100,50,25)],
    'classifier__activation': ['relu', 'tanh', 'logistic'],
    'classifier__alpha': [0.0001, 0.001, 0.01],
}

# Create a pipeline with scaling if needed
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), feature_names),  # Scale numeric features if needed
        # You can add other transformers for categorical features if needed
    ])

pipeline = Pipeline([
#     ('preprocessor', preprocessor),
    ('classifier', mlp)
])

# Create a StratifiedKFold cross-validator
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform grid search with cross-validation and compute F1 score for each fold
f1_scorer = make_scorer(f1_score, average='binary')  # 'binary' for binary classification
grid_search = GridSearchCV(pipeline, param_grid, scoring=f1_scorer, cv=cv, verbose=1, n_jobs=-1)
grid_search.fit(X, y)

# Print the best parameters and the corresponding F1 score
print("Best Parameters: ", grid_search.best_params_)
print("Best F1 Score: ", grid_search.best_score_)

# Get the best model from the grid search
best_model = grid_search.best_estimator_

# Compute F1 score for each fold using cross_val_score
f1_scores = cross_val_score(best_model, X, y, cv=cv, scoring=f1_scorer)

# Create a DataFrame to store F1 scores for each fold
f1_df = pd.DataFrame({'Fold': range(1, len(f1_scores)+1), 'F1 Score': f1_scores})

# Display the DataFrame
f1_df

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best Parameters:  {'classifier__activation': 'logistic', 'classifier__alpha': 0.001, 'classifier__hidden_layer_sizes': (100, 50, 25)}
Best F1 Score:  0.6752675359100018


,Fold,F1 Score
0,1,0.652174
1,2,0.623853
2,3,0.661088
3,4,0.695971
4,5,0.679070


## LightGBM

In [38]:
# Define the LGBMClassifier
lgbm = LGBMClassifier()

# Define the parameter grid for grid search
param_grid = {
    'classifier__learning_rate': [0.001, 0.01, 0.1],
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [3, 5, 7],
}

# Create a pipeline with scaling if needed
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), feature_names),  # Scale numeric features if needed
        # You can add other transformers for categorical features if needed
    ])

pipeline = Pipeline([
#     ('preprocessor', preprocessor),
    ('classifier', lgbm)
])

# Create a StratifiedKFold cross-validator
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform grid search with cross-validation and compute F1 score for each fold
f1_scorer = make_scorer(f1_score, average='binary')  # 'binary' for binary classification
grid_search = GridSearchCV(pipeline, param_grid, scoring=f1_scorer, cv=cv, verbose=1, n_jobs=-1)
grid_search.fit(X, y)

# Print the best parameters and the corresponding F1 score
print("Best Parameters: ", grid_search.best_params_)
print("Best F1 Score: ", grid_search.best_score_)

# Get the best model from the grid search
best_model = grid_search.best_estimator_

# Compute F1 score for each fold using cross_val_score
f1_scores = cross_val_score(best_model, X, y, cv=cv, scoring=f1_scorer)

# Create a DataFrame to store F1 scores for each fold
f1_df = pd.DataFrame({'Fold': range(1, len(f1_scores)+1), 'F1 Score': f1_scores})

# Display the DataFrame
f1_df

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[LightGBM] [Info] Number of positive: 614, number of negative: 688
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000419 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4500
[LightGBM] [Info] Number of data points in the train set: 1302, number of used features: 33
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.471582 -> initscore=-0.113794
[LightGBM] [Info] Start training from score -0.113794
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

[LightGBM] [Info] Number of positive: 492, number of negative: 550
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000399 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4422
[LightGBM] [Info] Number of data points in the train set: 1042, number of used features: 33
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.472169 -> initscore=-0.111440
[LightGBM] [Info] Start training from score -0.111440
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

,Fold,F1 Score
0,1,0.713178
1,2,0.744186
2,3,0.720000
3,4,0.736462
4,5,0.769841
